In [28]:

from Bio import SeqIO
import pandas as pd

In [29]:
!mmseqs

/bin/bash: /home/anwer/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
MMseqs2 (Many against Many sequence searching) is an open-source software suite for very fast, 
parallelized protein sequence searches and clustering of huge protein sequence data sets.

Please cite: M. Steinegger and J. Soding. MMseqs2 enables sensitive protein sequence searching for the analysis of massive data sets. Nature Biotechnology, doi:10.1038/nbt.3988 (2017).

MMseqs2 Version: 13.45111
© Martin Steinegger (martin.steinegger@snu.ac.kr)

usage: mmseqs <command> [<args>]

Easy workflows for plain text input/output
  easy-search       	Sensitive homology search
  easy-cluster      	Slower, sensitive clustering
  easy-linclust     	Fast linear time cluster, less sensitive clustering
  easy-taxonomy     	Taxonomic classification
  easy-rbh          	Find reciprocal best hit

Main workflows for database input/output
  search            	Sensitive homology search
  map        

# Cluster data
First step in making a independent test set is to cluster all sequences at a low identity like 30% or smaler.
This will ensure that you minimize the data leakage between your training data set and your test data set.

The identity value of 30% comes from the field of structural prediction.
https://tbiomed.biomedcentral.com/articles/10.1186/s12976-015-0014-1
https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-2932-0

In [30]:
## cluster data
!mmseqs easy-cluster my_fasta.fasta clu_30 tmp --min-seq-id 0.3 -c 0.8 --cov-mode 1

/bin/bash: /home/anwer/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
easy-cluster my_fasta.fasta clu_30 tmp --min-seq-id 0.3 -c 0.8 --cov-mode 1 

MMseqs Version:                     	13.45111
Substitution matrix                 	nucl:nucleotide.out,aa:blosum62.out
Seed substitution matrix            	nucl:nucleotide.out,aa:VTML80.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	2147483647
Alphabet size                       	nucl:5,aa:21
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.8
Coverage mode                       	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask low

[=================================================================] 100.00% 24.01K 0s 39ms


Sort kmer 0h 0m 0s 55ms
Sort by rep. sequence 0h 0m 0s 12ms
Time for fill: 0h 0m 0s 1ms
Time for merging to pref: 0h 0m 0s 0ms
Time for processing: 0h 0m 0s 145ms
rescorediagonal tmp/16873760946489227705/input tmp/16873760946489227705/input tmp/16873760946489227705/clu_tmp/821555203396869019/linclust/14723821866126630999/pref tmp/16873760946489227705/clu_tmp/821555203396869019/linclust/14723821866126630999/pref_rescore1 --sub-mat nucl:nucleotide.out,aa:blosum62.out --rescore-mode 0 --wrapped-scoring 0 --filter-hits 0 -e 0.001 -c 0.8 -a 0 --cov-mode 1 --min-seq-id 0.5 --min-aln-len 0 --seq-id-mode 0 --add-self-matches 0 --sort-results 0 --db-load-mode 0 --threads 16 --compressed 0 -v 3 

[=================================================================] 100.00% 24.01K 0s 18ms     
Time for merging to pref_rescore1: 0h 0m 0s 4ms==================>] 98.81% 23.72K eta 0s       
Time for processing: 0h 0m 0s 50ms
clust tmp/16873760946489227705/input tmp/16873760946489227705/clu_tmp/8215552

Index table k-mer threshold: 154 at k-mer size 6 
Index table: counting k-mers


[=================================================================] 100.00% 16.45K 0s 104ms    
Index table: Masked residues: 224397
Index table: fill
[=================================================================] 100.00% 16.45K 0s 102ms    =================>                                          ] 35.15% 5.78K eta 0s       
Index statistics
Entries:          3168075
DB size:          506 MB
Avg k-mer size:   0.049501
Top 10 k-mers
    MADYIC	36
    PMFLLS	32
    IHDKNI	28
    KASCHL	28
    QVIFRR	27
    GPGGTL	25
    RYACLR	25
    WFNRWR	25
    LHPYLS	24
    TFDAFS	23
Time for index table init: 0h 0m 0s 550ms
Process prefiltering step 1 of 1

k-mer similarity threshold: 154
Starting prefiltering scores calculation (step 1 of 1)
Query db start 1 to 16448
Target db start 1 to 16448
[=================================================================] 100.00% 16.45K 0s 227ms    

2.526002 k-mers per position
293 DB matches per sequence
0 overflows
0 queries produce too many hits (t

Index table k-mer threshold: 118 at k-mer size 6 
Index table: counting k-mers
[=================================================================] 100.00% 13.03K 0s 92ms     
Index table: Masked residues: 170815
Index table: fill
[=================================================================] 100.00% 13.03K 0s 113ms    
Index statistics
Entries:          5005075
DB size:          516 MB
Avg k-mer size:   0.078204
Top 10 k-mers
    LLLLLL	44
    SSSSSS	30
    LLLLSL	25
    LLLLAL	21
    LLLLLV	21
    LPLLLL	19
    AALLLL	18
    LGLLLL	18
    PLLLLL	18
    LLSLLL	18
Time for index table init: 0h 0m 0s 515ms
Process prefiltering step 1 of 1

k-mer similarity threshold: 118
Starting prefiltering scores calculation (step 1 of 1)
Query db start 1 to 13030
Target db start 1 to 13030
[=================================================================] 100.00% 13.03K 4s 495ms    

165.918714 k-mers per position
4666 DB matches per sequence
0 overflows
0 queries produce too many hits (truncat

# Pick n largest clusters

Depending on the size you can choose n to minimize the loss of data while still be a good representation of your data set.

In [34]:
#create test set out of top 10 clusters

! cat clu_30_cluster.tsv | cut -f 1 | uniq -c | sort -n | awk ' { t = $1; $1 = $2; $2 = t; print; } ' | cut -f 1 -d " " > test_clusters.tsv


/bin/bash: /home/anwer/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [19]:
# Create test sequences
! for CLU in `cat test_clusters.tsv`; do grep ${CLU} clu_30_cluster.tsv ; done > test_sequences.tsv

/bin/bash: /home/anwer/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


# Extract test sequences

All sequences in the clusters selected for test set should be extracted from the training data set.

In [24]:
## Read test_tsv

df = pd.read_csv("test_clusters.tsv", sep = "\t")
df

,PRO_13858
0,PRO_12288
1,PRO_5903
2,PRO_18020
3,PRO_19549
4,PRO_3624
5,PRO_14121
6,PRO_13679
7,PRO_137
8,PRO_17619
9,PRO_136


In [15]:
ids = list(df["sequence"])
file = "non_redundant_v2.fasta"
dict_seq = {"ID": [], "Seq": [], "Temp": []}
for rec in SeqIO.parse(file, "fasta"):
    dict_seq["ID"].append(rec.id)
    dict_seq["Seq"].append(str(rec.seq))
    dict_seq["Temp"].append(float(rec.description.split()[-1]))
    
df_seq = pd.DataFrame(dict_seq)

df_test=pd.merge(df, df_seq, how='left', left_on=['sequence'], right_on=['ID'])
df_train=pd.merge(df_seq, df_test, how='outer', indicator=True, left_on=['ID'], right_on=['ID']).query('_merge=="left_only"').drop('_merge', axis=1)    

In [16]:
## write train and test fasta

### Write train
with open("train.fasta", "w") as file_writer:
    for row in df_train.iterrows():

        file_writer.write(f">{row[1]['ID']} {row[1]['Temp_x']}\n{row[1]['Seq_x']}\n")
        
### Write train
with open("test.fasta", "w") as file_writer:
    for row in df_test.iterrows():
        file_writer.write(f">{row[1]['ID']} {row[1]['Temp']}\n{row[1]['Seq']}\n")
        


2719862